# Convert COCO to Pascal VOC

This Python script is designed to convert annotations from the COCO format (stored in .json files) to the Pascal VOC format (stored in .xml files).

This script converts COCO-format annotations (from .json files) into Pascal VOC-format annotations (stored in .xml files). It extracts image metadata and bounding boxes from COCO .json files, reformats them, and saves them in an XML format compatible with Pascal VOC. 

## Pascal VOC format for Object Detection with Landing Lens
Landing Lens Object Detection projects, can work with the Pascal VOC (Visual Object Classes) format. This format involves uploading the original (unlabeled) image and a corresponding XML file. The XML file contains the label (annotation) details of its paired image. The XML file essentially tells LandingLens where each label is on its associated image and what the name of the class is.

The image and XML file must have the same file name (with different extensions). For example:

*vehicle_123.png
*vehicle_123.xml

Additional details at:
https://support.landing.ai/docs/upload-labeled-images-od?highlight=pascal%20voc

## Imports

* pycocotools.coco.COCO: This is a library used to work with COCO (Common Objects in Context) datasets, which contain image annotations in .json format.
* os: Provides functions to interact with the file system (e.g., checking paths, creating directories).
* lxml.etree, lxml.objectify: These are used to create and manage XML documents for Pascal VOC annotations.
* shutil: Used for file operations like deleting or copying directories.
* tqdm: A library that adds a progress bar to loops, providing visual feedback.
* sys: Interacts with the Python interpreter, useful for command-line argument handling.
* argparse: Used to parse command-line arguments.


In [ ]:
from pycocotools.coco import COCO
import os
from lxml import etree, objectify
import shutil
from tqdm import tqdm
import sys
import argparse

## Functions

### catid2name()

catid2name(coco)

Converts the category IDs from the COCO dataset to category names (a dictionary that maps IDs to names).
Input: A COCO object loaded from the .json file.
Output: A dictionary where keys are category IDs, and values are category names (e.g., {1: 'person', 2: 'bicycle', ...}).

In [ ]:
def catid2name(coco):
    classes = dict()
    for cat in coco.dataset['categories']:
        classes[cat['id']] = cat['name']
    return classes

### save_anno_to_xml()

save_anno_to_xml(filename, size, objs, save_path)

Parameters:
* filename: The name of the image file.
* size: A dictionary containing the width, height, and depth (usually 3 for RGB images) of the image.
* objs: A list of objects, where each object contains the category name and bounding box coordinates (xmin, ymin, xmax, ymax).
save_path: The directory where the resulting XML file will be saved.

Converts image annotation information into Pascal VOC format and saves it as an XML file. Uses objectify.ElementMaker to create XML elements. Annotations include information about the image file, the source, image size, and the list of objects.
For each object (category and bounding box), it appends the corresponding <object> element to the XML tree.
The XML file is saved with the same name as the image but with the .xml extension.

In [ ]:

def save_anno_to_xml(filename, size, objs, save_path):
    E = objectify.ElementMaker(annotate=False)
    anno_tree = E.annotation(
        E.folder("DATA"),
        E.filename(filename),
        E.source(
            E.database("The VOC Database"),
            E.annotation("PASCAL VOC"),
            E.image("flickr")
        ),
        E.size(
            E.width(size['width']),
            E.height(size['height']),
            E.depth(size['depth'])
        ),
        E.segmented(0)
    )
    for obj in objs:
        E2 = objectify.ElementMaker(annotate=False)
        anno_tree2 = E2.object(
            E.name(obj[0]),
            E.pose("Unspecified"),
            E.truncated(0),
            E.difficult(0),
            E.bndbox(
                E.xmin(obj[1]),
                E.ymin(obj[2]),
                E.xmax(obj[3]),
                E.ymax(obj[4])
            )
        )
        anno_tree.append(anno_tree2)
    anno_path = os.path.join(save_path, filename[:-3] + "xml")
    etree.ElementTree(anno_tree).write(anno_path, pretty_print=True)


## load_coco()

load_coco(anno_file, xml_save_path)

Parameters:
* anno_file: Path to the COCO .json annotation file.
* xml_save_path: Directory where the converted XML files will be saved.

Loads the COCO dataset from a .json file, extracts annotations, and saves them in VOC format. Loads the COCO dataset using COCO(anno_file). Calls catid2name to create a mapping of category IDs to category names. Iterates over all image IDs (imgIds) in the COCO dataset.

For each image:
1) Extracts the image dimensions and file name.
2) Loads the annotations (anns) for that image, including bounding boxes (bbox).
3) Converts bounding boxes from the COCO format (x, y, width, height) to Pascal VOC format (xmin, ymin, xmax, ymax).
4) Calls save_anno_to_xml to save the annotations in Pascal VOC XML format.

In [ ]:
def load_coco(anno_file, xml_save_path):
    if os.path.exists(xml_save_path):
        shutil.rmtree(xml_save_path)
    os.makedirs(xml_save_path)

    coco = COCO(anno_file)
    classes = catid2name(coco)
    imgIds = coco.getImgIds()
    classesIds = coco.getCatIds()
    for imgId in tqdm(imgIds):
        size = {}
        img = coco.loadImgs(imgId)[0]
        filename = img['file_name']
        width = img['width']
        height = img['height']
        size['width'] = width
        size['height'] = height
        size['depth'] = 3
        annIds = coco.getAnnIds(imgIds=img['id'], iscrowd=None)
        anns = coco.loadAnns(annIds)
        objs = []
        for ann in anns:
            object_name = classes[ann['category_id']]
            # bbox:[x,y,w,h]
            bbox = list(map(int, ann['bbox']))
            xmin = bbox[0]
            ymin = bbox[1]
            xmax = bbox[0] + bbox[2]
            ymax = bbox[1] + bbox[3]
            obj = [object_name, xmin, ymin, xmax, ymax]
            objs.append(obj)
        save_anno_to_xml(filename, size, objs, xml_save_path)


### parseJsonFile()

parseJsonFile(data_dir, xmls_save_path)

Parameters:

* data_dir: Path to either a COCO .json file or a directory containing multiple datasets.
* xmls_save_path: Path where the resulting XML files will be saved.

Determines whether the input is a directory or a single file, then processes it accordingly. If the input is a directory, it processes both the train2017 and val2017 subsets by loading their corresponding .json files and saving the converted annotations.
If the input is a single file, it processes that file directly using load_coco.

In [ ]:
def parseJsonFile(data_dir, xmls_save_path):
    assert os.path.exists(data_dir), "data dir:{} does not exits".format(data_dir)

    if os.path.isdir(data_dir):
        data_types = ['train2017', 'val2017']
        for data_type in data_types:
            ann_file = 'instances_{}.json'.format(data_type)
            xmls_save_path = os.path.join(xmls_save_path, data_type)
            load_coco(ann_file, xmls_save_path)
    elif os.path.isfile(data_dir):
        anno_file = data_dir
        load_coco(anno_file, xmls_save_path)

## Main Script Block

Handles command-line argument parsing and script execution.

Uses argparse to define two command-line arguments:
--data-dir (-d): Path to the COCO .json annotation file or directory.
--save-path (-s): Path where the XML files should be saved.

If command-line arguments are provided, it calls parseJsonFile with the specified paths.
If no arguments are provided, it uses default paths (./data/labels/coco/train.json and ./data/convert/voc) to process the dataset.


In [ ]:
if __name__ == '__main__':
    """
    Script Description:
        This script is used to convert json files in coco format to xml files in voc format
    Parameter Description:
        data_dir:path of the json file
        xml_save_path:path of the xml output.
    """

    parser = argparse.ArgumentParser()
    parser.add_argument('-d', '--data-dir', type=str, default='./data/labels/coco/train.json', help='json path')
    parser.add_argument('-s', '--save-path', type=str, default='./data/convert/voc', help='xml save path')
    opt = parser.parse_args()
    print(opt)

    if len(sys.argv) > 1:
        parseJsonFile(opt.data_dir, opt.save_path)
    else:
        data_dir = './data/labels/coco/train.json'
        xml_save_path = './data/convert/voc'
        parseJsonFile(data_dir=data_dir, xmls_save_path=xml_save_path)